Sentiment Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("IMDB Dataset.csv\IMDB Dataset.csv")

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.shape

type(data)

pandas.core.frame.DataFrame

In [5]:
data["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

As we can see that we need to the one hot encoding

positive > 1
negative < 1

In [6]:
data.replace({"sentiment":{"positive":1, "negative":0}}, inplace=True)

In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


Now our data has been cleaned, we need to train our model and this can be done with the help of the RNN and the LSTM concepts

In [8]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

c:\Users\VAIBHAVRAI\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [10]:
train_data.shape

(40000, 2)

In [11]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [22]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test =  pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [23]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]])

In [24]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]])

In [25]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

Now we are going to build out the LSTM model

In [32]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [35]:
model.fit(X_train, Y_train, epochs = 5, batch_size=4, validation_split = 0.2)

Epoch 1/5
8000/8000 [==============================] - 199s 24ms/step - loss: 0.4067 - accuracy: 0.8098 - val_loss: 0.2643 - val_accuracy: 0.8888
Epoch 2/5
8000/8000 [==============================] - 178s 22ms/step - loss: 0.2331 - accuracy: 0.9088 - val_loss: 0.2708 - val_accuracy: 0.8905
Epoch 3/5
8000/8000 [==============================] - 171s 21ms/step - loss: 0.1855 - accuracy: 0.9291 - val_loss: 0.2850 - val_accuracy: 0.8924
Epoch 4/5
8000/8000 [==============================] - 178s 22ms/step - loss: 0.1408 - accuracy: 0.9476 - val_loss: 0.3214 - val_accuracy: 0.8910
Epoch 5/5
8000/8000 [==============================] - 183s 23ms/step - loss: 0.1021 - accuracy: 0.9638 - val_loss: 0.3510 - val_accuracy: 0.8836


In [36]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 [==============================] - 4s 11ms/step - loss: 0.3339 - accuracy: 0.8856


In [37]:
model.save("sentimodel.h5")

In [38]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

Building a predective system

In [58]:
def predictive_system(review):
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequences = pad_sequences(sequences, maxlen=200)
    pridiction = model.predict(padded_sequences)
    sentiment = "positive" if pridiction[0][0] > 0.5 else "negative"
    return sentiment

In [59]:
predictive_system("hello this movie is very bad")

1/1 [==============================] - 0s 30ms/step


'negative'

In [60]:
predictive_system("hello this movie is very good")

1/1 [==============================] - 0s 19ms/step


'positive'